In [1]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Config, MT5EncoderModel, MT5Tokenizer, Trainer, TrainingArguments
from progeny_tokenizer import TAPETokenizer
import numpy as np
import math
import random
import scipy
import time
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, Dataset, BatchSampler
import typing
from pathlib import Path
import argparse
from collections import OrderedDict
import pickle
import pathlib

import matplotlib.pyplot as plt

In [2]:
noperturb_results_tsv = 'generated_seqs/basegen_SST5/SST5_leave4out3keep200_t5base_basegen_lre-04_25ep/25Kgen_SST5_leave4out3keep200_t5base_basegen_lre-04_25ep-basegen_seqs25000.tsv'

In [3]:
perturb_results_tsvs = [
#     'generated_seqs/basegen_SST5/SST5_leave4out_small_basegen_lre-04_25ep/25Kgen_SST5_leave4out_small_basegen_lre-04_25ep-basegen_seqs25000.tsv',
#     'generated_seqs/basegen_SST5/SST5_leave4out_t5base_basegen_lre-04_25ep/25Kgen_SST5_leave4out_t5base_basegen_lre-04_25ep-basegen_seqs25000.tsv',
#     'generated_seqs/basegen_SST5/SST5_leave4out3keep200_small_basegen_lre-04_25ep/25Kgen_SST5_leave4out3keep200_small_basegen_lre-04_25ep-basegen_seqs25000.tsv',
    'generated_seqs/basegen_SST5/SST5_leave4out3keep200_t5base_basegen_lre-04_25ep/25Kgen_SST5_leave4out3keep200_t5base_basegen_lre-04_25ep-basegen_seqs25000.tsv',
#     'generated_seqs/basegen_SST5/SST5_leave34out_small_basegen_lre-04_25ep/25Kgen_SST5_leave34out_small_basegen_lre-04_25ep-basegen_seqs25000.tsv',
    'generated_seqs/basegen_SST5/SST5_leave34out_t5base_basegen_lre-04_25ep/25Kgen_SST5_leave34out_t5base_basegen_lre-04_25ep-basegen_seqs25000.tsv'

]

In [4]:
noperturb_df = pd.read_table(noperturb_results_tsv)

In [5]:
noperturb_df = noperturb_df.sort_values(by='disc_pred', ascending=False)
noperturb_df

,disc_pred,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,generated_seq
0,14.238190,3,0.999955,1,0.000024,0.999963,41.360153,"It's an upbeat, funny, wry adventure that puts..."
1,14.159341,3,0.996055,1,0.000147,0.999042,115.663300,A dark comedy that's surprisingly charming.
2,14.156757,4,0.999906,1,0.000036,0.999913,46.189896,"A great, well-crafted film about one man's inn..."
3,14.152386,4,0.999930,1,0.000037,0.999938,37.665958,"An uplifting, romantic comedy that gives its a..."
4,14.149242,4,0.998888,1,0.000076,0.999635,40.508041,It has both the charm and the humor of a good ...
...,...,...,...,...,...,...,...,...
25408,-13.505506,0,0.999880,0,0.999956,0.000033,84.890915,As much if every tactic is to minimize the eff...
25409,-13.521099,0,0.999716,0,0.999950,0.000044,73.198692,The most ridiculous way to describe a movie as...
25410,-13.530113,0,0.999951,0,0.999959,0.000034,28.555031,Unwilling to give any of his characters the ti...
25411,-13.539289,0,0.999937,0,0.999944,0.000045,18.774836,"... the movie is a mess, full of clichés, and ..."


In [6]:
# noperturb_df['sent_delta'] = noperturb_df['gt_class_pred'] - noperturb_df['gen_input_seq_class']
# noperturb_df

# Compute % target class for noperturb

In [7]:
target_classes = [3,4]
topk_list = [10000, 1000, 100, 10]

percent_target_class = []

In [8]:
gt_class_preds = noperturb_df['gt_class_pred']
# gen_input_seq_classes = noperturb_df['gen_input_seq_class']
# sent_deltas = noperturb_df['sent_delta']

In [9]:
df = noperturb_df

In [10]:
for target_class in target_classes:
    total_num  = len(df['gt_class_pred'])
    print("target_class: ", target_class)
    num_target_class = np.sum(df['gt_class_pred'] == target_class)
    
    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)
    
    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
#         topk_sent_deltas = sent_deltas[:topk]
        
        
        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
#         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)
    
#         topk_sent_delta_mean = np.mean(topk_sent_deltas)
#         print("topk_sent_deltas: ", topk_sent_deltas)
#         print("topk_sent_delta_mean: ", topk_sent_delta_mean)
        
        print("*")
    print("--------------")
    
    
print("-------For all target classes-------")
print("target_classes: ", target_classes)
total_num  = len(df['gt_class_pred'])
num_target_class = np.sum(df['gt_class_pred'].isin(target_classes))

percent_target_class = num_target_class / total_num *100
print("percent_target_class: ", percent_target_class)

for topk in topk_list:
    topk_gt_class_preds = gt_class_preds[:topk]
#     topk_sent_deltas = sent_deltas[:topk]


    topk_num  = len(topk_gt_class_preds)
    print("topk: ", topk)
#         print("topk_gt_class_preds: ", topk_gt_class_preds)
    topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
    topk_percent_target_class = topk_num_target_class / topk_num *100
#     print("topk_num_target_class: ", topk_num_target_class)
#     print("topk_num: ", topk_num)
    print("topk_percent_target_class: ", topk_percent_target_class)

#     topk_sent_delta_mean = np.mean(topk_sent_deltas)
#         print("topk_sent_deltas: ", topk_sent_deltas)
#     print("topk_sent_delta_mean: ", topk_sent_delta_mean)

    print("*")

target_class:  3
percent_target_class:  10.467083776020147
topk:  10000
topk_percent_target_class:  24.84
*
topk:  1000
topk_percent_target_class:  63.9
*
topk:  100
topk_percent_target_class:  35.0
*
topk:  10
topk_percent_target_class:  30.0
*
--------------
target_class:  4
percent_target_class:  1.7904222248455515
topk:  10000
topk_percent_target_class:  4.390000000000001
*
topk:  1000
topk_percent_target_class:  26.700000000000003
*
topk:  100
topk_percent_target_class:  64.0
*
topk:  10
topk_percent_target_class:  70.0
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target_class:  12.2575060008657
topk:  10000
topk_percent_target_class:  29.23
*
topk:  1000
topk_percent_target_class:  90.60000000000001
*
topk:  100
topk_percent_target_class:  99.0
*
topk:  10
topk_percent_target_class:  100.0
*


# iterate through all perturbed result tsv files

In [14]:
for perturb_results_tsv in perturb_results_tsvs:
    print("*-"*30)
    print("perturb_results_tsv: ", perturb_results_tsv)
    
    perturb_df = pd.read_table(perturb_results_tsv)
    perturb_df = perturb_df.sort_values(by='disc_pred', ascending=False)
#     perturb_df['sent_delta'] = perturb_df['gt_class_pred'] - perturb_df['gen_input_seq_class']

    gt_class_preds = perturb_df['gt_class_pred']
#     gen_input_seq_classes = perturb_df['gen_input_seq_class']
#     sent_deltas = perturb_df['sent_delta']

    generated_seq_ppls = perturb_df['generated_seq_ppl']

    for target_class in target_classes:
        total_num  = len(perturb_df['gt_class_pred'])
        print("target_class: ", target_class)
        num_target_class = np.sum(perturb_df['gt_class_pred'] == target_class)

        percent_target_class = num_target_class / total_num *100
        print("percent_target_class: ", percent_target_class)

        for topk in topk_list:
            topk_gt_class_preds = gt_class_preds[:topk]
#             topk_sent_deltas = sent_deltas[:topk]

            topk_num  = len(topk_gt_class_preds)
            print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
            topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
            topk_percent_target_class = topk_num_target_class / topk_num *100
#             print("topk_num_target_class: ", topk_num_target_class)
#             print("topk_num: ", topk_num)
            print("topk_percent_target_class: ", topk_percent_target_class)

#             topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#             print("topk_sent_delta_mean: ", topk_sent_delta_mean)

            print("*")
        print("--------------")


    print("-------For all target classes-------")
    print("target_classes: ", target_classes)
    total_num  = len(perturb_df['gt_class_pred'])
    num_target_class = np.sum(perturb_df['gt_class_pred'].isin(target_classes))

    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)

    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
#         topk_sent_deltas = sent_deltas[:topk]

        topk_generated_seq_ppls = generated_seq_ppls[:topk]

        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)

    
        topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
        topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
        print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
        print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)
#         topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#         print("topk_sent_delta_mean: ", topk_sent_delta_mean)

        print("*")
    
    
    
        
    # E[% positive, strong-positive] computation

    df = perturb_df

    num_rounds = 100 # N
    round_pool_size = 1000
    topk = 100 # K
    main_pool_size = 25000
    target_classes = [3, 4]

    round_topk = {}
#     cols_to_sort = ['latent_head_pred']
    cols_to_sort = ['disc_pred']
    df_main_pool = df.sample(n=main_pool_size)


    print("--------------")
    print("E[% positive, strong-positive] computation")
    # print("Sorted by ", cols_to_sort)
    for col_to_sort in cols_to_sort:
        print("col_to_sort: ", col_to_sort)
        round_topk[col_to_sort] = {}
        for round_ind in range(num_rounds):
            sampled_rows = df_main_pool.sample(n=round_pool_size)
            sorted_sampled_rows = sampled_rows.sort_values(by=col_to_sort, ascending=False)[:topk]
            topk_rows = sorted_sampled_rows[:topk]
            round_topk[col_to_sort][round_ind] = {}



            for target_class in target_classes:
                total_num  = len(topk_rows['gt_class_pred'])
    #             print("target_class: ", target_class)
                num_target_class = np.sum(topk_rows['gt_class_pred'] == target_class)

                percent_target_class = num_target_class / total_num *100
    #             print("percent_target_class: ", percent_target_class)

                round_topk[col_to_sort][round_ind][target_class] = percent_target_class



    #         print("target_classes: ", target_classes)
            total_num  = len(topk_rows['gt_class_pred'])
            num_target_class = np.sum(topk_rows['gt_class_pred'].isin(target_classes))

            percent_target_class = num_target_class / total_num *100
    #         print("percent_target_class: ", percent_target_class)
            round_topk[col_to_sort][round_ind]['all'] = percent_target_class


        for target_class in target_classes:
            percent_values = []
            for round_ind in range(num_rounds):
                percent_values.append(round_topk[col_to_sort][round_ind][target_class])
            print("target_class: ", target_class)
            mean_percent_values = np.mean(percent_values)
            std_percent_values = np.std(percent_values)
            print("mean_percent_values: ", mean_percent_values)
            print("std_percent_values: ", std_percent_values)



        percent_values = []
        for round_ind in range(num_rounds):
            percent_values.append(round_topk[col_to_sort][round_ind]['all'])
        print("target_classes: ", target_classes)
        mean_percent_values = np.mean(percent_values)
        std_percent_values = np.std(percent_values)
        print("mean_percent_values: ", mean_percent_values)
        print("std_percent_values: ", std_percent_values)

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
perturb_results_tsv:  generated_seqs/basegen_SST5/SST5_leave4out3keep200_t5base_basegen_lre-04_25ep/25Kgen_SST5_leave4out3keep200_t5base_basegen_lre-04_25ep-basegen_seqs25000.tsv
target_class:  3
percent_target_class:  10.467083776020147
topk:  10000
topk_percent_target_class:  24.84
*
topk:  1000
topk_percent_target_class:  63.9
*
topk:  100
topk_percent_target_class:  35.0
*
topk:  10
topk_percent_target_class:  30.0
*
--------------
target_class:  4
percent_target_class:  1.7904222248455515
topk:  10000
topk_percent_target_class:  4.390000000000001
*
topk:  1000
topk_percent_target_class:  26.700000000000003
*
topk:  100
topk_percent_target_class:  64.0
*
topk:  10
topk_percent_target_class:  70.0
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target_class:  12.2575060008657
topk:  10000
topk_percent_target_class:  29.23
topk_generated_seq_ppl_mean:  65.75631463816224
topk_generated_s